In [1]:
%matplotlib inline

import os
import glob

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import keras
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, BatchNormalization, \
                         Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, LearningRateScheduler, \
                            ReduceLROnPlateau
import keras.backend as K

Using TensorFlow backend.


In [2]:
data_map = {}
max_class = 100
for i,f in enumerate(glob.glob("data/*.npy")):
    if i < max_class:
        data_map[os.path.basename(f)] = np.load(f)

train_samples_per_class = 2**10
test_samples_per_class = 2**8
train_data_map, test_data_map = {}, {}
for key in data_map:
    train_data_map[key] = data_map[key].reshape((-1, 28, 28))[:,:,:,np.newaxis][:train_samples_per_class]
    test_data_map[key] = data_map[key].reshape((-1, 28, 28))[:,:,:,np.newaxis][train_samples_per_class:train_samples_per_class+test_samples_per_class]
    train_data_map[key] = train_data_map[key].astype("float32") / 255
    test_data_map[key] = test_data_map[key].astype("float32") / 255

class_orders = data_map.keys()
train_data = np.concatenate([train_data_map[key] for key in class_orders])
test_data = np.concatenate([test_data_map[key] for key in class_orders])
train_target = np.concatenate([train_data_map[k].shape[0]*[i]
                               for i,k in enumerate(class_orders)])
test_target = np.concatenate([test_data_map[k].shape[0]*[i]
                              for i,k in enumerate(class_orders)])
train_target = np_utils.to_categorical(train_target)
test_target = np_utils.to_categorical(test_target)

In [3]:
print data_map.keys()
print train_data.shape, train_target.shape, test_data.shape, test_target.shape

['leg.npy', 'keyboard.npy', 'mermaid.npy', 'strawberry.npy', 'dolphin.npy', 'nail.npy', 'vase.npy', 'bus.npy', 'trumpet.npy', 'elephant.npy', 'television.npy', 'bracelet.npy', 'basketball.npy', 't-shirt.npy', 'banana.npy', 'sleeping bag.npy', 'eraser.npy', 'lion.npy', 'hourglass.npy', 'tennis racquet.npy', 'chandelier.npy', 'airplane.npy', 'shovel.npy', 'hamburger.npy', 'matches.npy', 'pool.npy', 'broccoli.npy', 'drill.npy', 'diamond.npy', 'guitar.npy', 'hat.npy', 'duck.npy', 'lighter.npy', 'book.npy', 'piano.npy', 'toothpaste.npy', 'mountain.npy', 'car.npy', 'postcard.npy', 'tractor.npy', 'shorts.npy', 'crocodile.npy', 'octagon.npy', 'beard.npy', 'drums.npy', 'clarinet.npy', 'oven.npy', 'The Great Wall of China.npy', 'bat.npy', 'hot air balloon.npy', 'anvil.npy', 'flashlight.npy', 'bench.npy', 'boomerang.npy', 'spider.npy', 'hot dog.npy', 'sweater.npy', 'barn.npy', 'computer.npy', 'bird.npy', 'sailboat.npy', 'bridge.npy', 'hockey stick.npy', 'roller coaster.npy', 'wine bottle.npy', 'f

In [4]:
num_class = len(data_map)

train_data_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                                    height_shift_range=0.1, horizontal_flip=True,
                                    vertical_flip=False)
test_data_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                                   height_shift_range=0.1, horizontal_flip=True,
                                   vertical_flip=False)
train_data_gen.fit(train_data)
test_data_gen.fit(test_data)

In [5]:
input_layer = Input(shape=(28, 28, 1), name='Input_layer')

# convblock 01
x = Conv2D(32, (3, 3), padding='same', activation='relu',
           input_shape=input_layer.shape, name='Conv01_layer')(input_layer)
x = Dropout(0.05, name='Dropout06')(x)
x = BatchNormalization(name='BatchNormalization01_layer')(x)
x = Conv2D(32, (3, 3), activation='relu', name='Conv02_layer')(x)
x = Dropout(0.05, name='Dropout05')(x)
x = BatchNormalization(name='BatchNormalization02_layer')(x)
x = Conv2D(32, (3, 3), activation='relu', name='Conv03_layer')(x)
x = Dropout(0.05, name='Dropout04')(x)
x = BatchNormalization(name='BatchNormalization03_layer')(x)
x = MaxPooling2D(pool_size=(2, 2), name='MaxPool01_layer')(x)

# convblock 02
x = Conv2D(64, (3, 3), padding='same', activation='relu',
           input_shape=input_layer.shape, name='Conv04_layer')(x)
x = Dropout(0.05, name='Dropout03')(x)
x = BatchNormalization(name='BatchNormalization04_layer')(x)
x = Conv2D(64, (3, 3), activation='relu', name='Conv05_layer')(x)
x = Dropout(0.05, name='Dropout02')(x)
x = BatchNormalization(name='BatchNormalization05_layer')(x)
x = MaxPooling2D(pool_size=(2, 2), name='MaxPool02_layer')(x)

# fully connected dense block
x = Flatten(name='Flatten_layer')(x)
x = Dense(512, activation='relu', name='Dense01_layer')(x)
x = BatchNormalization(name='BatchNormalization06_layer')(x)
x = Dense(num_class, name='logits_layer')(x)
x = Dropout(0.1, name='Dropout01')(x)
output = Activation('softmax', name='Softmax_layer')(x)

In [6]:
model = Model(input_layer, output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 28, 28, 1)         0         
_________________________________________________________________
Conv01_layer (Conv2D)        (None, 28, 28, 32)        320       
_________________________________________________________________
Dropout06 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
BatchNormalization01_layer ( (None, 28, 28, 32)        128       
_________________________________________________________________
Conv02_layer (Conv2D)        (None, 26, 26, 32)        9248      
_________________________________________________________________
Dropout05 (Dropout)          (None, 26, 26, 32)        0         
_________________________________________________________________
BatchNormalization02_layer ( (None, 26, 26, 32)        128       
__________

In [7]:
learning_rate = 0.03
decay = 0.2
batch_size = 2**8
epochs = 100

callbacks = [EarlyStopping(monitor='loss', min_delta=0.001, patience=10),
             LearningRateScheduler(lambda epoch:learning_rate/(1 + decay * epoch)),
             ReduceLROnPlateau(monitor='loss')]
             
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
K.set_value(model.optimizer.lr, learning_rate)
model.fit_generator(train_data_gen.flow(train_data, train_target, batch_size=batch_size),
                    validation_data=test_data_gen.flow(test_data, test_target, batch_size=batch_size),
                    steps_per_epoch=train_data.shape[0] // batch_size,
                    validation_steps=test_data.shape[0] // batch_size,
                    epochs=epochs,
                    workers=8,
                    callbacks=callbacks)

Epoch 1/100
400/400 [==============================] - 41s - loss: 2.9684 - acc: 0.3140 - val_loss: 3.9346 - val_acc: 0.2387
Epoch 2/100
400/400 [==============================] - 40s - loss: 2.1998 - acc: 0.4704 - val_loss: 2.1232 - val_acc: 0.4639
Epoch 3/100
400/400 [==============================] - 39s - loss: 1.9651 - acc: 0.5236 - val_loss: 1.7432 - val_acc: 0.5542
Epoch 4/100
400/400 [==============================] - 39s - loss: 1.8192 - acc: 0.5581 - val_loss: 2.9539 - val_acc: 0.4873
Epoch 5/100
400/400 [==============================] - 40s - loss: 1.7366 - acc: 0.5774 - val_loss: 1.6049 - val_acc: 0.6023
Epoch 6/100
400/400 [==============================] - 40s - loss: 1.6722 - acc: 0.5904 - val_loss: 1.6182 - val_acc: 0.5834
Epoch 7/100
400/400 [==============================] - 40s - loss: 1.6210 - acc: 0.6020 - val_loss: 1.3824 - val_acc: 0.6421
Epoch 8/100
400/400 [==============================] - 39s - loss: 1.5711 - acc: 0.6129 - val_loss: 1.4053 - val_acc: 0.6384


400/400 [==============================] - 39s - loss: 1.1094 - acc: 0.7116 - val_loss: 0.9740 - val_acc: 0.7525
Epoch 67/100
400/400 [==============================] - 39s - loss: 1.1034 - acc: 0.7136 - val_loss: 0.9557 - val_acc: 0.7550
Epoch 68/100
400/400 [==============================] - 39s - loss: 1.0967 - acc: 0.7145 - val_loss: 0.9750 - val_acc: 0.7512
Epoch 69/100
400/400 [==============================] - 39s - loss: 1.0998 - acc: 0.7133 - val_loss: 0.9744 - val_acc: 0.7531
Epoch 70/100
400/400 [==============================] - 39s - loss: 1.1036 - acc: 0.7125 - val_loss: 0.9703 - val_acc: 0.7568
Epoch 71/100
400/400 [==============================] - 39s - loss: 1.0974 - acc: 0.7151 - val_loss: 0.9988 - val_acc: 0.7476
Epoch 72/100
400/400 [==============================] - 39s - loss: 1.0941 - acc: 0.7163 - val_loss: 0.9810 - val_acc: 0.7477
Epoch 73/100
400/400 [==============================] - 39s - loss: 1.0976 - acc: 0.7139 - val_loss: 0.9844 - val_acc: 0.7525
Epoch